
Exercício 2

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

    1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é gcd(a,b) = min{r > 0 | E [s,t] . r = a * s + b * t}.
    2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
    3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
    4. Prove a correção  do programa LPA em ambas as codificações.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT,BVType
from pysmt.logics import QF_NIA
import itertools

1.

gcd(a,b) = min{r > 0 | E [s,t] . r = a*s+b*t}, entao temos de provar o seguinte:  
(r final = a * s + b * t) -> (r > 0)  
e que r final <= todos os r em que (r = a * s + b * t) -> (r > 0)  

In [2]:
def pos_condi(a, b, states):
    st = states[-1]
    
    # exists = ((∃s,t⋅r=a∗s+b∗t) -> (r >0)), prova r>0 ∣ ∃s,t⋅r=a∗s+b∗t
    exists =  Implies(Exists([st['s'], st['t']], Equals(st['r'], Plus(Times(Int(a), st['s']), Times(Int(b), st['t'])))), 
                          GT(st['r'], Int(0)))
    
    # Pós-condição
    r_condition = []
    for st in states:
        # exists -> (r <= all r{0,i}), prova o min{}
        r_condition.append(LE(states[-1]['r'], st['r']))
        
    return Implies(exists, And(*r_condition))

b)

## LPA?
1. {Pre-condição: a > 0 and b > 0 and a < N and b < N}  
2. r, r', s, s', t, t' := a, b, 1, 0, 0, 1  
3. while r' != 0  
4.   {Invariante de Ciclo: r = a * s' + b * t' and r' = a * s + b * t}  
5.   q := r div r'  
6.   r, r', s, s', t, t' := r', r - q * r', s', s - q * s', t', t - q * t'  
7. {Pós-condição: r = mdc(a, b) and r = a * s + b * t}  
8. {Pós-condição Forte: r = mdc(a, b) and r = a * s + b * t and r = gcd(a, b) and s * a + t * b = mdc(a, b)}  

In [3]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from pysmt.logics import QF_LIA

def euclidean_extended(a, b, N):
    # Variáveis
    r = Symbol("r", INT)
    r_prime = Symbol("r_prime", INT)
    s = Symbol("s", INT)
    s_prime = Symbol("s_prime", INT)
    t = Symbol("t", INT)
    t_prime = Symbol("t_prime", INT)
    q = Symbol("q", INT)
    
    # Condições iniciais
    assume = And(a > 0, b > 0, a < N, b < N)
    
    # Inicialização
    pre_conditions = And(r == a, r_prime == b, s == 1, s_prime == 0, t == 0, t_prime == 1)
    
    # Comando havoc
    havoc_command = Or(Implies(Not(r_prime == 0), r == r_prime - q * r_prime),
                      Implies(r_prime == 0, r == r_prime))
    
    # Pós-condição
    post_condition = Or(Implies(Not(r_prime == 0), And(r_prime * s == a, r_prime * t == b)),
                       Implies(r_prime == 0, And(r == a, s == 1, t == 0)))
    
    # Verificando se a pós-condição é válida
    assertion = Implies(pre_conditions, And(havoc_command, post_condition))
    solver = get_model(assertion)
    
    if solver:
        print("Programa válido")
        print("Modelo encontrado:")
        for var in [r, r_prime, s, s_prime, t, t_prime, q]:
            print(f"{var}: {solver.get_value(var)}")
    else:
        print("Programa inválido")

# Exemplo de uso
a = 35
b = 15
N = 100

#euclidean_extended(a, b, N)

In [3]:
n=40
a=10
b=30


def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)

    state['r'] = Symbol('r'+str(i),INT)
    state['r_'] = Symbol('r_'+str(i),INT)
    state['s'] = Symbol('s'+str(i),INT)
    state['s_'] = Symbol('s_'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['t_'] = Symbol('t_'+str(i),INT)
    state['q'] = Symbol('q' +str(i),INT)

    return state
     
def init(state):
    E = Equals(state['pc'],Int(0))

    R = Equals(state['r'],Int(a))
    R_ = Equals(state['r_'],Int(b))
    S = Equals(state['s'], Int(1))
    S_ = Equals(state['s_'],Int(0))
    T = Equals(state['t'],Int(0))
    T_ = Equals(state['t_'],Int(1))
    Q = Equals(state['q'],Int(0))

    return And(E,R,R_,S,S_,T,T_)

def trans(curr,prox):

    t01 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r_'],Int(0))), Equals(prox['pc'],Int(1)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t02 = And(Equals(curr['pc'],Int(0)), Equals(curr['r_'],Int(0)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t10 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(0)),
              Equals(curr['q'], Div(curr['r'],curr['r_'])),
              Equals(prox['r'],curr['r_']), Equals(prox['r_'], Minus(curr['r'],Times(curr['q'],curr['r_']))),
              Equals(prox['s'],curr['s_']), Equals(prox['s_'], Minus(curr['s'],Times(curr['q'],curr['s_']))),
              Equals(prox['t'],curr['t_']), Equals(prox['t_'], Minus(curr['t'],Times(curr['q'],curr['t_']))))

    t22 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    return Or(t01,t02,t10,t22)

def euclides(declare,init,trans,k):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(k)]
        
        post = pos_condi(a, b, states)
        solver.add_assertion(post)
        
        solver.add_assertion(init(states[0]))
        solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))
        
        if solver.solve():
            print('pos-condicao comprovada!')
            print(f'gdc({a},{b}) = {solver.get_value(states[-1]["r"])}')
        else:
            print('pos-condicao falsa')

euclides(declare,init,trans,n)

pos-condicao comprovada!
gdc(10,30) = 10
